<a href="https://colab.research.google.com/github/hamzafarooq/multi-agent-course/blob/main/Module_3_Agentic_RAG/Agentic_RAG_with_Semantic_Cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic RAG with Semantic Cache

This notebook combines two powerful concepts:

- **Semantic Cache** — A FAISS-backed cache that stores previous query embeddings and their answers. When a new query is semantically similar to a cached one, the stored answer is returned instantly — no LLM or API call needed.
- **Agentic RAG** — An intelligent retrieval system that routes queries to the right knowledge source: OpenAI documentation (via Qdrant), 10-K financial filings (via Qdrant), or live internet search (via SerpApi).

## Architecture

```
User Query
    │
    ▼
┌─────────────────────────────┐
│  Is query time-sensitive?   │  ──YES──▶  Agentic RAG (no caching)
│  (current events, "today",  │
│   live data, etc.)          │
└──────────────┬──────────────┘
               │ NO
               ▼
┌─────────────────────────────┐
│   Semantic Cache Lookup     │  ──HIT──▶  Return cached answer ⚡
│   (FAISS similarity search) │
└──────────────┬──────────────┘
               │ MISS
               ▼
┌─────────────────────────────┐
│      Agentic RAG Router     │
│   (GPT-4o classifies query) │
└──────┬──────────┬───────────┘
       │          │           │
  OPENAI      10K_DOC    INTERNET
  QUERY       QUERY       QUERY
    │            │            │
  Qdrant      Qdrant      SerpApi
  (RAG)       (RAG)      (live web)
       │          │           │
       └──────────┴───────────┘
               │
               ▼
    Store answer in cache 💾
               │
               ▼
          Return answer
```

## Why combine them?

- **Speed**: Cached answers return in milliseconds vs. 2–5 seconds for full RAG.
- **Cost**: Fewer LLM and API calls for repeated or similar questions.
- **Correctness**: Time-sensitive queries (e.g., *"What happened today?"*) always bypass the cache to ensure fresh answers.

## 1. Setup

Install dependencies, clone the course repository (which contains `rag_helpers.py` and the pre-built Qdrant vector database), and import the helper module.

In [ ]:
!pip install -U faiss-cpu sentence_transformers transformers openai qdrant_client python-dotenv nest_asyncio -q

import os, sys, nest_asyncio

# ── Colab: clone the course repo if not already present ──────────────────────
try:
    import google.colab
    _REPO = "/content/multi-agent-course"
    if not os.path.exists(_REPO):
        os.system(f"git clone https://github.com/hamzafarooq/multi-agent-course.git {_REPO}")
        print("Repository cloned ✅")
    else:
        print("Repository already present ✅")
    _MODULE_DIR = f"{_REPO}/Module_3_Agentic_RAG"
except ImportError:
    # Running locally — rag_helpers.py lives in Module_3_Agentic_RAG/
    _MODULE_DIR = os.path.dirname(os.path.abspath("__file__"))
    print(f"Running locally — helpers path: {_MODULE_DIR}")

sys.path.insert(0, _MODULE_DIR)
nest_asyncio.apply()  # Required for asyncio.run() inside Jupyter/Colab

from rag_helpers import init_rag, SemanticCaching, agentic_rag_with_cache
print("✅ Helpers imported from rag_helpers.py")

## 2. API Keys

**On Google Colab** — store keys in the Secrets panel (`🔑` icon, left sidebar):
| Secret name | Where to get it |
|---|---|
| `SERP_API_KEY` | [serpapi.com](https://serpapi.com) |
| `OPENAI_API_KEY` | [platform.openai.com](https://platform.openai.com) |

**Running locally** — add keys to `Module_3_Agentic_RAG/.env`:
```
serp_api_key=<your_key>
openai_api_key=<your_key>
```
The cell below detects the environment automatically and loads from the right source.

In [ ]:
# ── Load API keys ─────────────────────────────────────────────────────────────
try:
    from google.colab import userdata
    serp_api_key   = userdata.get('SERP_API_KEY')
    openai_api_key = userdata.get('OPENAI_API_KEY')
    QDRANT_PATH    = f"{_REPO}/Module_3_Agentic_RAG/Agentic_RAG/qdrant_data"
    print("Colab: credentials loaded from Secrets.")
except ImportError:
    from dotenv import load_dotenv, find_dotenv
    load_dotenv(find_dotenv())
    serp_api_key   = os.getenv("serp_api_key")   or os.getenv("SERP_API_KEY")
    openai_api_key = os.getenv("openai_api_key") or os.getenv("OPENAI_API_KEY")
    QDRANT_PATH    = os.path.join(_MODULE_DIR, "Agentic_RAG", "qdrant_data")
    print("Local: credentials loaded from .env.")

print(f"SerpApi key:    {'✅' if serp_api_key else '❌ MISSING'}")
print(f"OpenAI API key: {'✅' if openai_api_key else '❌ MISSING'}")

# ── Initialise the RAG pipeline (loads models + connects to Qdrant) ───────────
init_rag(openai_api_key=openai_api_key, serp_api_key=serp_api_key, qdrant_path=QDRANT_PATH)

## 3. Create the Semantic Cache

`SemanticCaching` is defined in `rag_helpers.py`. It provides:

| Method | Purpose |
|---|---|
| `is_time_sensitive(q)` | Returns `True` for questions with temporal keywords — these always bypass the cache |
| `check_cache(q)` | Embeds the query and runs a FAISS nearest-neighbour search; returns hit/miss + pre-computed embedding |
| `add_to_cache(q, answer, embedding)` | Persists a new entry to FAISS + JSON after a RAG call |

**Similarity threshold** (`threshold=0.2`): distance ≤ 0.2 (Euclidean) counts as a hit. Lower = stricter matching. Try `0.1` for exact-ish matches or `0.35` for a looser hit rate.

In [16]:
# Instantiate the semantic cache
# Set clear_on_init=True to wipe any previously stored entries
cache = SemanticCaching(json_file='rag_cache.json', threshold=0.2, clear_on_init=True)

Loading Nomic embedding model...


Embedding model ready.
Semantic cache cleared.


## 4. Agentic RAG Pipeline (from `rag_helpers.py`)

The pipeline components are all defined in `rag_helpers.py` — see the file for full implementations.

| Function | What it does |
|---|---|
| `get_internet_content(query)` | Live Google search via SerpApi |
| `route_query(query)` | GPT-4o classifies into `OPENAI_QUERY`, `10K_DOCUMENT_QUERY`, or `INTERNET_QUERY` |
| `_retrieve_and_respond(query, action)` | Embeds query → searches the right Qdrant collection → generates a cited RAG answer |
| `_run_rag_pipeline(query)` | Orchestrates routing + handler dispatch; returns the answer string |
| **`agentic_rag_with_cache(query, cache)`** | **Main entry point** — applies the cache layer on top of the full pipeline |

**Qdrant collections loaded by `init_rag()`:**
- `opnai_data` — OpenAI Agents documentation  
- `10k_data`   — Uber 2021 & Lyft 2024 10-K filings

## 5. Demo — Semantic Cache + Agentic RAG in Action

`agentic_rag_with_cache(query, cache)` is the single function to call. It handles all routing, retrieval, caching, and display automatically.

### Test queries — three cache paths

| Query | Expected path |
|---|---|
| *"What was Uber's revenue in 2021?"* | Cache MISS → 10K RAG → stored |
| *"How much did Uber earn in 2021?"* | Cache HIT (semantically similar) |
| *"How do I build an agent with the OpenAI Agents SDK?"* | Cache MISS → OpenAI RAG → stored |
| *"What are the best AI tools this week?"* | Time-sensitive → bypass cache → SerpApi |
| *"What is the current stock price of Apple?"* | Time-sensitive → bypass cache → SerpApi |
| *"What are the most popular open-source LLMs?"* | Cache MISS → INTERNET → SerpApi → stored |

In [20]:
# Test 1: Cache MISS — should route to 10K_DOCUMENT_QUERY and store result
result = agentic_rag_with_cache("What was Uber's revenue in 2021?")

👤 User Query: What was Uber's revenue in 2021?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: 10K_DOCUMENT_QUERY
📝 Reason: The question relates to Uber's financials.

💾 Answer cached for future similar queries.

🤖 Response:
Uber's revenue in 2021 was $3,208,323,000 [1].



In [21]:
# Test 2: Cache HIT — semantically similar to Test 1, returns instantly from cache
result = agentic_rag_with_cache("How much did Uber earn in fiscal year 2021?")

👤 User Query: How much did Uber earn in fiscal year 2021?

✅ Semantic Cache HIT (row 0, similarity: 0.838, time: 0.123s)

🤖 Response (cached):
Uber's revenue in 2021 was $3,208,323,000 [1].



In [22]:
# Test 3: Cache MISS — routes to OPENAI_QUERY and stores result
result = agentic_rag_with_cache("How do I build an agent with the OpenAI Agents SDK?")

👤 User Query: How do I build an agent with the OpenAI Agents SDK?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: OPENAI_QUERY
📝 Reason: Question about OpenAI's documentation on agents.

💾 Answer cached for future similar queries.

🤖 Response:
To build an agent with the OpenAI Agents SDK, you need to focus on three core components: Model, Tools, and Instructions. Here's a step-by-step guide using the SDK:

1. **Model Selection**: Choose an appropriate LLM for your agent by evaluating task complexity, latency, and cost. Start with the most capable model to establish a performance baseline, then experiment with smaller models to optimize for cost and speed [1][2].

2. **Define Tools**: Tools are external functions or APIs the agent can utilize to perform actions. Define these tools clearly for the agent to use, such as a `get_weather` function in the example provided [1].

3. **Provide Instructions**: Write explicit guidelines and guardrails that define agent behavior.

In [23]:
# Test 4: Time-sensitive query — BYPASSES cache, calls Ares API for live answer
result = agentic_rag_with_cache("What are the best AI tools this week?")

👤 User Query: What are the best AI tools this week?

⏰ Time-sensitive query detected — bypassing semantic cache to ensure a fresh answer.

📍 Route: INTERNET_QUERY
📝 Reason: General inquiry about AI tools.
Getting your response from the internet 🌐 ...

🤖 Response (live):
[1] The Best AI Tools for 2026
    Without a doubt, ChatGPT, Gemini, and Claude are the best AI tools to date. They can provide answers to your everyday questions, do web searches ...
    Source: https://medium.com/artificial-corner/the-best-ai-tools-for-2026-933535a44f8b

[2] I tried 70+ best AI tools in 2026
    I went deep into each tool, from image generation to email automation, chatbot building to scheduling assistants.
    Source: https://www.techradar.com/best/best-ai-tools

[3] I Tested 47 AI Tools in 30 Days — Here Are the Only 7 Worth ...
    I Tested 47 AI Tools in 30 Days — Here Are the Only 7 Worth Paying For How I cut my workday from 10 hours to 6 without sacrificing quality ...
    Source: https://levelu

In [24]:
# Test 5: Time-sensitive query — stock price, always fetched live
result = agentic_rag_with_cache("What is the current stock price of Apple?")

👤 User Query: What is the current stock price of Apple?

⏰ Time-sensitive query detected — bypassing semantic cache to ensure a fresh answer.

📍 Route: INTERNET_QUERY
📝 Reason: Question on real-time stock price
Getting your response from the internet 🌐 ...

🤖 Response (live):
[1] Stock Price - Apple - Investor Relations
    Stock Quote: NASDAQ: AAPL ; Day's Open262.60 ; Closing Price260.58 ; Volume30.8 ; Intraday High264.48 ; Intraday Low260.05.
    Source: https://investor.apple.com/stock-price/default.aspx

[2] AAPL: Apple Inc - Stock Price, Quote and News
    Apple Inc AAPL:NASDAQ ; Open258.97 ; Day High264.75 ; Day Low258.16 ; Prev Close260.58 ; 52 Week High288.62 ...
    Source: https://www.cnbc.com/quotes/AAPL

[3] Buy or Sell Apple Stock - AAPL Stock Price Quote & News
    Shares are currently priced at $261.38, which is +1.2% above the low and -1.3% below the high. Apple(AAPL) shares are trading with a volume of 42.05M, against a ...
    Source: https://robinhood.com/us/en/stoc

In [25]:
# Test 6: Cache MISS — INTERNET_QUERY, stored after Ares API call
result = agentic_rag_with_cache("What are the most popular open-source LLMs?")

👤 User Query: What are the most popular open-source LLMs?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: INTERNET_QUERY
📝 Reason: General inquiry about open-source LLMs.
Getting your response from the internet 🌐 ...

💾 Answer cached for future similar queries.

🤖 Response:
[1] Open LLM Leaderboard 2025
    This LLM leaderboard displays the latest public benchmark performance for SOTA open-sourced model versions released after April 2024.
    Source: https://www.vellum.ai/open-llm-leaderboard

[2] Top 10 open source LLMs for 2025
    Top open source LLMs in 2024 · 1. LLaMA 3 · 2. Google Gemma 2 · 3. Command R+ · 4. Mistral-8x22b · 5. Falcon 2 · 6. Grok 1.5 · 7. Qwen1.5 · 8. BLOOM.
    Source: https://www.instaclustr.com/education/open-source-ai/top-10-open-source-llms-for-2025/

[3] I locally benchmarked 41 open-source LLMs across 19 ...
    Hello everyone! I benchmarked 41 open-source LLMs using lm-evaluation-harness. Here are the 19 tasks covered:.
    Source: http

In [26]:
# Test 7: Cache HIT — similar to Test 6
result = agentic_rag_with_cache("Which open-source large language models are most widely used?")

👤 User Query: Which open-source large language models are most widely used?

❌ Semantic Cache MISS — running Agentic RAG pipeline...

📍 Route: INTERNET_QUERY
📝 Reason: Question on general open-source LLMs.
Getting your response from the internet 🌐 ...

💾 Answer cached for future similar queries.

🤖 Response:
[1] Top 10 open source LLMs for 2025
    Unlike proprietary models developed by companies like OpenAI and Google, open source LLMs are licensed to be freely used, modified, and distributed by anyone.
    Source: https://www.instaclustr.com/education/open-source-ai/top-10-open-source-llms-for-2025/

[2] 9 Top Open-Source LLMs for 2026 and Their Uses
    LLM are the foundation models of popular and widely-used chatbots, like ChatGPT and Google Gemini. ... most powerful open-source large language ...
    Source: https://www.datacamp.com/blog/top-open-source-llms

[3] The best open source large language model
    The largest open-source models, DeepSeek-V3 and DeepSeek-R1, match GPT-4o

## 6. Inspect the Cache

View all entries currently stored in the semantic cache.

In [27]:
print(f"Total cached entries: {len(cache.cache['questions'])}")
print(f"FAISS index size: {cache.index.ntotal}\n")

for i, (q, a) in enumerate(zip(cache.cache['questions'], cache.cache['response_text'])):
    print(f"[{i}] Q: {q}")
    print(f"    A: {a[:120]}...\n" if len(a) > 120 else f"    A: {a}\n")

Total cached entries: 4
FAISS index size: 4

[0] Q: What was Uber's revenue in 2021?
    A: Uber's revenue in 2021 was $3,208,323,000 [1].

[1] Q: How do I build an agent with the OpenAI Agents SDK?
    A: To build an agent with the OpenAI Agents SDK, you need to focus on three core components: Model, Tools, and Instructions...

[2] Q: What are the most popular open-source LLMs?
    A: [1] Open LLM Leaderboard 2025
    This LLM leaderboard displays the latest public benchmark performance for SOTA open-so...

[3] Q: Which open-source large language models are most widely used?
    A: [1] Top 10 open source LLMs for 2025
    Unlike proprietary models developed by companies like OpenAI and Google, open s...



## Assignment: Extend the System

Try one or more of these extensions:

1. **Adjustable similarity threshold** — Experiment with `threshold=0.1` (stricter) vs `threshold=0.35` (looser). How does it affect hit rate and answer quality?

2. **Cache TTL (Time-To-Live)** — Add an expiry timestamp to each cache entry. Stale entries (e.g., older than 7 days) should be evicted and re-fetched.

3. **Sub-query division** — Before checking the cache, use a GPT call to split compound questions (e.g., *"What was Uber and Lyft revenue in 2021?"*) into sub-queries. Check and populate the cache per sub-query.

4. **Cache analytics** — Track and display cache hit rate, average latency for hits vs misses, and the most-queried topics over a session.